In [ ]:
# 설치
! pip install smolagents
# 마지막 릴리스 대신 소스에서 설치하려면 위 명령을 주석 처리하고 다음 명령의 주석을 해제하십시오.
# ! pip install git+https://github.com/huggingface/smolagents.git

# 다중 에이전트 시스템 조정하기 🤖🤝🤖

이 노트북에서는 **다중 에이전트 웹 브라우저, 즉 웹을 사용하여 문제를 해결하기 위해 여러 에이전트가 협력하는 에이전트 시스템**을 만들 것입니다!

간단한 계층 구조가 될 것입니다.

```
              +----------------+
              | 관리자 에이전트  |
              +----------------+
                       |
        _______________|______________
       |                              |
코드 인터프리터            +------------------+
    도구                    | 웹 검색 에이전트 |
                            +------------------+
                               |            |
                        웹 검색 도구     |
                                   웹 페이지 방문 도구
```
이 시스템을 설정해 봅시다.

아래 줄을 실행하여 필요한 종속성을 설치하십시오.

In [ ]:
!pip install smolagents[toolkit] --upgrade -q

추론 공급자를 호출하기 위해 HF에 로그인합시다.

In [ ]:
from huggingface_hub import login

login()

⚡️ 우리 에이전트는 HF의 추론 API를 사용하는 `InferenceClientModel` 클래스를 사용하여 [Qwen/Qwen2.5-Coder-32B-Instruct](https://huggingface.co/Qwen/Qwen2.5-Coder-32B-Instruct)로 구동됩니다. 추론 API를 사용하면 모든 OS 모델을 빠르고 쉽게 실행할 수 있습니다.

> [!TIP]
> 추론 공급자는 서버리스 추론 파트너가 제공하는 수백 개의 모델에 대한 액세스를 제공합니다. 지원되는 공급자 목록은 [여기](https://huggingface.co/docs/inference-providers/index)에서 찾을 수 있습니다.

In [ ]:
model_id = "Qwen/Qwen2.5-Coder-32B-Instruct"

## 🔍 웹 검색 도구 만들기

웹 브라우징을 위해 Google 검색과 동등한 기능을 제공하는 기본 [WebSearchTool](https://huggingface.co/docs/smolagents/main/ko/reference/tools#smolagents.WebSearchTool) 도구를 이미 사용할 수 있습니다.

하지만 `WebSearchTool`에서 찾은 페이지를 살펴볼 수 있어야 합니다.
이를 위해 라이브러리에 내장된 `VisitWebpageTool`을 가져올 수 있지만, 어떻게 수행되는지 확인하기 위해 다시 빌드할 것입니다.

따라서 `markdownify`를 사용하여 `VisitWebpageTool` 도구를 처음부터 만들어 보겠습니다.

In [ ]:
import re
import requests
from markdownify import markdownify
from requests.exceptions import RequestException
from smolagents import tool


@tool
def visit_webpage(url: str) -> str:
    """지정된 URL의 웹 페이지를 방문하여 해당 내용을 마크다운 문자열로 반환합니다.

    Args:
        url: 방문할 웹 페이지의 URL입니다.

    Returns:
        마크다운으로 변환된 웹 페이지의 내용 또는 요청이 실패한 경우 오류 메시지입니다.
    """
    try:
        # URL로 GET 요청 보내기
        response = requests.get(url)
        response.raise_for_status()  # 잘못된 상태 코드에 대해 예외 발생

        # HTML 내용을 마크다운으로 변환
        markdown_content = markdownify(response.text).strip()

        # 여러 줄 바꿈 제거
        markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)

        return markdown_content

    except RequestException as e:
        return f"웹 페이지를 가져오는 중 오류 발생: {str(e)}"
    except Exception as e:
        return f"예기치 않은 오류 발생: {str(e)}"

자, 이제 도구를 초기화하고 테스트해 봅시다!

In [ ]:
print(visit_webpage("https://en.wikipedia.org/wiki/Hugging_Face")[:500])

## 다중 에이전트 시스템 구축하기 🤖🤝🤖

이제 `search` 및 `visit_webpage` 도구가 모두 있으므로 이를 사용하여 웹 에이전트를 만들 수 있습니다.

이 에이전트에 대해 어떤 구성을 선택해야 할까요?
- 웹 브라우징은 병렬 도구 호출이 필요 없는 단일 타임라인 작업이므로 JSON 도구 호출이 잘 작동합니다. 따라서 `ToolCallingAgent`를 선택합니다.
- 또한 웹 검색은 올바른 답변을 찾기 전에 많은 페이지를 탐색해야 하는 경우가 있으므로 `max_steps` 수를 10으로 늘리는 것을 선호합니다.

In [ ]:
from smolagents import (
    CodeAgent,
    ToolCallingAgent,
    InferenceClientModel,
    WebSearchTool,
    LiteLLMModel,
)

model = InferenceClientModel(model_id=model_id)

web_agent = ToolCallingAgent(
    tools=[WebSearchTool(), visit_webpage],
    model=model,
    max_steps=10,
    name="web_search_agent",
    description="웹 검색을 실행합니다.",
)

이 에이전트에 관리자 에이전트가 호출할 수 있도록 필수 속성인 `name`과 `description`을 부여했습니다.

그런 다음 관리자 에이전트를 만들고 초기화 시 `managed_agents` 인수에 관리되는 에이전트를 전달합니다.

이 에이전트는 계획과 사고를 담당하므로 고급 추론이 유익하므로 `CodeAgent`가 잘 작동합니다.

또한 현재 연도와 관련된 질문을 하고 추가 데이터 계산을 수행하려고 하므로 에이전트에 이러한 패키지가 필요한 경우를 대비하여 `additional_authorized_imports=["time", "numpy", "pandas"]`를 추가합니다.

In [ ]:
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent],
    additional_authorized_imports=["time", "numpy", "pandas"],
)

그게 다입니다! 이제 시스템을 실행해 봅시다! 계산과 조사가 모두 필요한 질문을 선택합니다.

In [ ]:
answer = manager_agent.run("LLM 교육이 2030년까지 현재 리듬으로 확장된다면 2030년까지 가장 큰 교육 실행에 필요한 전력은 GW 단위로 얼마일까요? 일부 국가와 비교하면 이는 무엇에 해당할까요? 사용된 모든 숫자에 대한 출처를 제공하십시오.")

답변으로 다음 보고서를 받습니다.
```
현재 성장 전망과 에너지 소비 추정치에 따르면 LLM 교육이 2030년까지 현재 리듬으로 계속 확장된다면 다음과 같습니다.

1. 2030년까지 가장 큰 교육 실행에 필요한 전력은 약 303.74GW이며, 이는 연간 약 2,660,762GWh에 해당합니다.

2. 이를 국가별 전력 소비량과 비교하면 다음과 같습니다.
   - 중국 전체 전력 소비량의 약 34%에 해당합니다.
   - 인도(184%), 러시아(267%), 일본(291%)의 전체 전력 소비량을 초과합니다.
   - 이탈리아나 멕시코와 같은 국가의 전력 소비량의 거의 9배에 달합니다.

3. 숫자 출처:
   - 향후 LLM 교육에 대한 초기 추정치 5GW는 AWS CEO Matt Garman으로부터 나왔습니다.
   - 성장 전망은 Springs의 시장 조사에서 CAGR 79.80%를 사용했습니다.
   - 국가별 전력 소비량 데이터는 주로 2021년 미국 에너지 정보국에서 가져왔습니다.
```

[확장 가설](https://gwern.net/scaling-hypothesis)이 계속 사실이라면 상당한 규모의 발전소가 필요할 것 같습니다.

우리 에이전트는 작업을 해결하기 위해 효율적으로 협력했습니다! ✅

💡 이 조정을 더 많은 에이전트로 쉽게 확장할 수 있습니다. 하나는 코드 실행을, 다른 하나는 웹 검색을, 다른 하나는 파일 로드를 처리합니다...